In [71]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [72]:
data_dir = 'color'

In [79]:
classes = ['ripe', 'unripe', 'defectiveDiseasedOverripe']
class_indices = {class_name: index for index, class_name in enumerate(classes)}

In [83]:
images = []
labels = []
for class_name in classes:
    class_dir = os.path.join(data_dir, class_name)
    for filename in os.listdir(class_dir):
        img_path = os.path.join(class_dir, filename)
        img = cv2.imread(img_path)
        
        # Check if the image was loaded successfully
        if img is None:
            print(f"Warning: Could not read image {img_path}. Skipping this file.")
            continue
        
        img = cv2.resize(img, (64, 64))
        img = img / 255.0
        images.append(img)
        labels.append(class_indices[class_name])

In [84]:
labels = tf.keras.utils.to_categorical(labels, num_classes=len(classes))


In [85]:
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)


In [86]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=len(classes), activation='softmax'))

In [87]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [88]:
model.fit(np.array(train_images), np.array(train_labels), epochs=25, validation_data=(np.array(test_images), np.array(test_labels)))

Epoch 1/25
4/4 [==============================] - 1s 170ms/step - loss: 1.5958 - accuracy: 0.4144 - val_loss: 1.4657 - val_accuracy: 0.6071
Epoch 2/25
4/4 [==============================] - 0s 71ms/step - loss: 1.1488 - accuracy: 0.6216 - val_loss: 0.6357 - val_accuracy: 0.7500
Epoch 3/25
4/4 [==============================] - 0s 78ms/step - loss: 0.5766 - accuracy: 0.6757 - val_loss: 0.6439 - val_accuracy: 0.6429
Epoch 4/25
4/4 [==============================] - 0s 78ms/step - loss: 0.3594 - accuracy: 0.8739 - val_loss: 0.4896 - val_accuracy: 0.7143
Epoch 5/25
4/4 [==============================] - 0s 100ms/step - loss: 0.3254 - accuracy: 0.8919 - val_loss: 0.3575 - val_accuracy: 0.8929
Epoch 6/25
4/4 [==============================] - 0s 94ms/step - loss: 0.1827 - accuracy: 0.9550 - val_loss: 0.3343 - val_accuracy: 0.8214
Epoch 7/25
4/4 [==============================] - 0s 82ms/step - loss: 0.1355 - accuracy: 0.9820 - val_loss: 0.3176 - val_accuracy: 0.8571
Epoch 8/25
4/4 [=========

In [91]:
def measure_size_and_density(image_path, mass):
    # Load the image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Warning: Could not read image {image_path}.")
        return None, None
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply a threshold to binarize the image
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)
    
    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Measure sizes (area) of each contour
    sizes = [cv2.contourArea(c) for c in contours]
    
    # Calculate density for each size
    densities = [(mass / size) if size > 0 else 0 for size in sizes]
    
    return sizes, densities
   # Example usage
data_dir = "size"  # Update this path to your image folder
mass = 5.0  # Example mass in grams for each cherry

# Iterate through each image in the folder
for filename in os.listdir(data_dir):
    img_path = os.path.join(data_dir, filename)
    sizes, densities = measure_size_and_density(img_path, mass)
    
    # Determine if the berries are defective based on density
    threshold_density = 0.5  # Example threshold; adjust as needed
    classifications = ["Defective" if density < threshold_density else "Good" for density in densities]
    
    print(f"Sizes for {filename}: {sizes}")
    print(f"Densities for {filename}: {densities}")
    print(f"Classifications for {filename}: {classifications}")


Sizes for 1.jpg: [0.0, 0.0, 0.0, 0.0, 0.0, 19954.0]
Densities for 1.jpg: [0, 0, 0, 0, 0, 0.0002505763255487622]
Classifications for 1.jpg: ['Defective', 'Defective', 'Defective', 'Defective', 'Defective', 'Defective']
Sizes for 2.jpg: [0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 83.5, 12847.5]
Densities for 2.jpg: [10.0, 0, 0, 0, 0, 0, 0, 0.059880239520958084, 0.0003891807744697412]
Classifications for 2.jpg: ['Good', 'Defective', 'Defective', 'Defective', 'Defective', 'Defective', 'Defective', 'Defective', 'Defective']
Sizes for 3.jpg: [11282.5]
Densities for 3.jpg: [0.00044316419233325947]
Classifications for 3.jpg: ['Defective']
Sizes for 4.jpg: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.5, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.5, 0.0, 8474.5]
Densities for 4.jpg: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.4, 0, 1.25, 0, 0, 0, 0, 10.0, 0, 0, 10.0, 0, 0.0005900053100477904]
Classifications for 4.jpg: ['Defective', 'Defective', 'Defective', 'De

In [92]:
model.save('coffee_cherry_classifier.h5')